# EB Part 06: Array Operations with NumPy

$$u^{n+1}_i = u^n_i-u^n_{i-1}$$

Now, given a vector $u^n = [0, 1, 2, 3, 4, 5]\ \ $   we can calculate the values of $u^{n+1}$ by iterating over the values of $u^n$ with a for loop.

In [1]:
import numpy

In [2]:
u = numpy.array((0 ,1 ,2, 3, 4, 5))

for i in range(1,len(u)):
    print(u[i] - u[i-1])

1
1
1
1
1


In this step all we have done is subtracted i-1 from i iteratively. But if we thought of this as a matrix, we could just subtract one vector from another...

In [3]:
# (2nd entry : end) - (1st entry : end -1)
u[1:] - u[0:-1]

array([1, 1, 1, 1, 1])

Computational gains are limited for a simple operation. But applied to more complex problems such as 2D linear convection the results are more pertinent.

In [6]:
nx = 81
ny = 81
nt = 100
c = 1
dx = 2 / (nx - 1)
dy = 2 / (ny -1)
sigma = .2
dt = sigma * dx

x = numpy.linspace(0,2,nx)
y = numpy.linspace(0,2,nx)

u = numpy.ones((ny,nx)) #create a 1 x n vector of 1's for u(n) and u(n+1)
un = numpy.ones((ny,nx))

### Assign initial conditions of 2
u[int(.5/dy):int(1/dy+1), int(0.5/dx):int(1/dx+1)] = 2

Now use the `%%timeit` function to see how long the nested loop method takes to execute...

In [9]:
%%timeit
u = numpy.ones((ny, nx))
u[int(.5 / dy): int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2

for n in range(nt+1):
    un = u.copy()
    row, col = u.shape # returns the dimensions of u[] and we are using these as our counts =ny,nx
    for j in range(1,row):
        for i in range(1,col):
            u[j,i] = (un[j,i] - (c*dt/dx * (un[j,i] - un[j,i-1])) - (c*dt/dy *(un[j,i] - un[j-1,i])) )
            
            u[0, :] = 1
            u[-1,:] = 1
            u[:, 0] = 1
            u[:,-1] = 1

3.1 s ± 113 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
# now with array operations
u = numpy.ones((ny,nx))
u[int(.5 / dy): int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2

for n in range(nt+1):
    un = u.copy()
    u[1:, 1:] = (un[1:,1:] - (c*dt/dx* (un[1:,1:] - un[1:,0:-1])) - (c*dt/dy* (un[1:,1:] - un[0:-1,1:])))
    
    u[0, :] = 1
    u[-1, :] = 1
    u[:, 0] = 1
    u[:, -1] = 1

4.66 ms ± 396 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The time has changes from *3.1 s* per loop to *4.66 ms* per loop by using array slicing! A significant performance improvement!